# Compulsory assignment 1 - Using Dask-ml on large data
## Group #16
### Thomas Moen and Jørgen Kongsro

In [1]:
%matplotlib notebook


import numpy as np
import pandas as pd
import dask
import os


from operator import itemgetter

import numpy as np
import scipy

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import matplotlib.pyplot as plt
import skimage.io


In [ ]:
# Install Kaggle API
# How to setup: https://github.com/Kaggle/kaggle-api
# or visit: https://adityashrm21.github.io/Setting-Up-Kaggle/

#!pip install kaggle

In [ ]:
# Download Kaggle data using Kaggle API
#!kaggle competitions download -c dat300-ca1-autumn-2019


In [ ]:
# Unzip Kaggle data
#!unzip "dat300-ca1-autumn-2019.zip" -d "/tmp/whatever"

In [2]:
# List files in directory

if os.name == 'nt':
    workdir = 'C://Users//thomoe//Documents//myDAT300//dat300-ca1-autumn-2019//'
elif os.name == 'posix':
    workdir = '/Users/jorgenkongsro/Downloads/dat300-ca1-autumn-2019/'
    
os.listdir(workdir)


['y_test_sampleSubmission.csv', 'X_train.csv', 'y_train.csv', 'X_test.csv']

In [31]:
def import_data(csv_file):
    """ Import data from csv file

    :param data: a .csv separated dataset
    :return: a pandas data array, df
    
    """
    
    
    df = dd.read_csv(csv_file)
    return df

x_train_df = import_data(workdir + 'X_train.csv')
x_test_df = import_data(workdir + 'X_test.csv')
y_train_df = import_data(workdir + 'y_train.csv')

x_test_df.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162
0,-0.180510,-0.192210,-0.20515,-0.215250,-0.215960,-0.215960,-0.216130,-0.218260,-0.22907,-0.506090,...,-0.31380,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709
1,-0.164560,-0.180510,-0.20515,-0.205680,-0.206210,-0.215250,-0.216130,-0.218260,-0.22907,-0.506090,...,-0.31380,-0.16006,-0.16006,-0.16006,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709,-0.16709
2,-0.044046,-0.050072,-0.05787,-0.066023,-0.067086,-0.067972,-0.087645,-0.099342,-0.30971,1.230800,...,-0.13746,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924,-0.10924
3,0.340010,0.329020,0.31236,0.239520,0.235090,0.189360,0.173410,0.161540,0.15835,-0.038613,...,0.38480,0.42301,0.42301,0.42301,0.32402,0.32402,0.32402,0.32402,0.32402,0.32402
4,-0.155170,-0.168990,-0.17927,-0.218790,-0.251050,-0.252110,-0.288620,-0.325130,-0.34622,-0.324840,...,-0.28911,-0.14978,-0.14978,-0.14978,-0.25570,-0.25570,-0.25570,-0.25570,-0.32433,-0.32433


In [30]:
def percent_missing(dataframe):
    """ Check for percent missing values in dataframe
    :param data: dataframe
    :return: dataframe
    
    """
    missing_values = dataframe.isnull().sum()
    
    with ProgressBar():
        percent_missing = ((missing_values / dataframe.index.size) * 100).compute()
        
    return percent_missing

check_for_missing(x_train_df)

[########################################] | 100% Completed | 37.5s


f1      3.628002
f2      3.754723
f3      3.836795
f4      4.133142
f5      4.299061
f6      4.377457
f7      9.171993
f8      9.276405
f9      9.410288
f10     3.628002
f11     3.754723
f12     3.836795
f13     4.133142
f14     4.299061
f15     4.377457
f16     9.171993
f17     9.276405
f18     9.410288
f19     3.628002
f20     3.754723
f21     3.836795
f22     4.133142
f23     4.299061
f24     4.377457
f25     9.171993
f26     9.276405
f27     9.410288
f28     3.628002
f29     3.754723
f30     3.836795
          ...   
f133    9.171993
f134    9.276405
f135    9.410288
f136    3.628002
f137    3.754723
f138    3.836795
f139    4.133142
f140    4.299061
f141    4.377457
f142    9.171993
f143    9.276405
f144    9.410288
f145    3.628002
f146    3.754723
f147    3.836795
f148    4.133142
f149    4.299061
f150    4.377457
f151    9.171993
f152    9.276405
f153    9.410288
f154    3.628002
f155    3.754723
f156    3.836795
f157    4.133142
f158    4.299061
f159    4.377457
f160    9.1719

In [33]:
def nan_filter(dataframe):
    """ Filter out NaN
    :param data: dataframe
    :return: dataframe
    
    """
    filtered = dataframe.dropna(how='all', subset=None, thresh=None).compute()

    return filtered

filtered = nan_filter(x_train_df)
percent_missing(filtered)

AttributeError: 'Series' object has no attribute 'compute'

In [ ]:
def drop_columns(dataframe):
    columns_to_drop = list(percent_missing[percent_missing >= 50].index)
    data = dataframe.drop(columns_to_drop, axis=1)

In [ ]:
def drop_rows:
    rows_to_drop = list(percent_missing[(percent_missing > 0) & (percent_missing < 5)].index)
    data_clean = data_clean.dropna(subset=rows_to_drop)

In [ ]:
def impute_rows:
    with ProgressBar():
    count_of_vehicle_colors = nyc_data_clean_stage1['Vehicle Color'].value_counts().compute()
most_common_color = count_of_vehicle_colors.sort_values(ascending=False).index[0]

# Fill missing vehicle color with the most common color
nyc_data_clean_stage2 = nyc_data_clean_stage1.fillna({'Vehicle Color': most_common_color})

In [28]:
# Print the size of a feature
print (x_train_df.count().compute())


f1      3041268
f2      3037269
f3      3034679
f4      3025327
f5      3020091
f6      3017617
f7      2866313
f8      2863018
f9      2858793
f10     3041268
f11     3037269
f12     3034679
f13     3025327
f14     3020091
f15     3017617
f16     2866313
f17     2863018
f18     2858793
f19     3041268
f20     3037269
f21     3034679
f22     3025327
f23     3020091
f24     3017617
f25     2866313
f26     2863018
f27     2858793
f28     3041268
f29     3037269
f30     3034679
         ...   
f133    2866313
f134    2863018
f135    2858793
f136    3041268
f137    3037269
f138    3034679
f139    3025327
f140    3020091
f141    3017617
f142    2866313
f143    2863018
f144    2858793
f145    3041268
f146    3037269
f147    3034679
f148    3025327
f149    3020091
f150    3017617
f151    2866313
f152    2863018
f153    2858793
f154    3041268
f155    3037269
f156    3034679
f157    3025327
f158    3020091
f159    3017617
f160    2866313
f161    2863018
f162    2858793
Length: 162, dtype: int6

In [ ]:
def percent_missing_values(data):
    """ Calculate the percent of missing values in the DataFrame
    
    :param data: pandas data frame
    :return: pandas data frame
    
    """
    missing_count = ((missing_values / df.index.size) * 100)
    return missing_count

missing

In [ ]:
def get_n_images(df):
    
    return len(df) - 1

nrows = get_n_images(xtrain_df)
print(nrows/8000)


In [ ]:
with ProgressBar():
    missing_count_pct = missing_count.compute()
missing_count_pct

In [ ]:
# View image data

plt.figure(figsize=(10, 10))
skimage.io.imshow()

In [ ]:
# Build model and train

# Split data into train and validation



from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from dask.diagnostics import ProgressBar

bern = BernoulliNB()
with ProgressBar():
    bern.fit(x_train_df, y_train_df)